# Starting and Stopping Simulations

Author: Sam Eure

May 28, 2025

### Create a new simulation

In [ ]:
import time

from air_sdk import AirApi
from air_sdk.endpoints import Simulation

api = AirApi.with_ngc_config()
# Or use SAK directly
# api = AirApi.with_api_key(api_key='nvapi-xyz')

json_data = {
    'format': 'JSON',
    'name': 'JSON Sim',
    'content': {
        'nodes': {
            'node1': {
                'cpu': 2,
                'memory': 1024,
                'storage': 10,
                'os': 'generic/ubuntu2204',
                'cpu_arch': 'x86',
            },
        },
        'oob': False,
    },
}

sim: Simulation = api.simulations.import_from_data(**json_data)
while sim.state == 'IMPORTING':
    sim.refresh()
    time.sleep(1)

### Start the simulation
Request that the simulation be started. If successful, the simulation will transition from `INACTIVE` -> `REQUESTING` -> `PROVISIONING` -> `BOOTING` -> `ACTIVE`.

In [7]:
sim.start()
while sim.state in {'REQUESTING', 'PROVISIONING', 'BOOTING'}:
    time.sleep(1)
    sim.refresh()

sim.state

'ACTIVE'

### Shut down the simulation

Request that the simulation be shut down. If successful, the simulation will transition from `ACTIVE` -> `PREPARE_SHUTDOWN` -> `SHUTTING_DOWN` -> `INACTIVE`.

In [8]:
sim.shutdown()
while sim.state in {'PREPARE_SHUTDOWN', 'SHUTTING_DOWN'}:
    time.sleep(1)
    sim.refresh()

for history in list(sim.get_history()):
    print(history.description)

Simulation `JSON Sim` with ID `c0b5e717-9bd1-46dd-84c3-fdd323b32cb2` created by importing a topology in `JSON` format. Topology is scheduled for validation.
Initiating topology validation.
Validation complete. Initiating creation of simulation objects.
Topology successfully imported.
Transitioning from `IMPORTING` to `INACTIVE` state.
Transitioning from `INACTIVE` to `REQUESTING` state.
Transitioning from `REQUESTING` to `PROVISIONING` state.
Transitioning from `PROVISIONING` to `PREPARE_BOOT` state.
Transitioning from `PREPARE_BOOT` to `BOOTING` state.
Transitioning from `BOOTING` to `ACTIVE` state.
Transitioning from `ACTIVE` to `PREPARE_SHUTDOWN` state.
Transitioning from `PREPARE_SHUTDOWN` to `SHUTTING_DOWN` state.
Transitioning from `SHUTTING_DOWN` to `INACTIVE` state.


In [9]:
sim.state

'INACTIVE'